In [1]:
# Import required libraries and module for data preprocessing
import pandas as pd
import numpy as np
import soundfile as sf
import librosa
import os

In [19]:
# Function for extracting and combining labels from .lab files into a DataFrame
def load_ultrasuite_labels(src_directory, src_dataset):
    
    directory = src_directory + src_dataset + '/word_labels/lab/'
    columns = ['start_time', 'end_time', 'utterance']
    all_labels_df = pd.DataFrame()

    for filename in os.listdir(directory):
    
        filepath = directory + filename
    
        labels_df = pd.read_csv(filepath, sep=" ", header=None, names=columns)
    
        # Extract the speaker, session and speech data from the filename and add to the dataframe
        labels_df['dataset'] = src_dataset
        labels_df['speaker'] = filename[0:3]
        if len(filename[4:-9]) == 0:
            labels_df['session'] = None
        else:
            labels_df['session'] = filename[4:-9]
        labels_df['speech_waveform'] = filename[-8:-4]

        # Tidy up data formatting and correct time based units
        labels_df['utterance'] = labels_df['utterance'].str.lower()
        labels_df['start_time'] = pd.to_timedelta(labels_df['start_time'] * 100)
        labels_df['end_time'] = pd.to_timedelta(labels_df['end_time'] * 100)

        # Append incoming labels to existing dataframe
        all_labels_df = all_labels_df.append(labels_df, ignore_index=True)
    
    return all_labels_df

In [20]:
# Load the labels for the Ultrax Speech Sound Disorders dataset
uxssd_df = load_ultrasuite_labels('data/ultrasuite/labels-uxtd-uxssd-upx/', 'uxssd')

In [21]:
# Preview the data
uxssd_df.head()

,start_time,end_time,utterance,dataset,speaker,session,speech_waveform
0,00:00:01.340000,00:00:02.040000,th,uxssd,02M,BL1,069B
1,00:00:02.460000,00:00:03.350000,atha,uxssd,02M,BL1,069B
2,00:00:03.790000,00:00:04.650000,eethee,uxssd,02M,BL1,069B
3,00:00:05.210000,00:00:06.110000,otho,uxssd,02M,BL1,069B
4,00:00:00.970000,00:00:01.480000,core,uxssd,04M,Maint1,017A


In [22]:
# Load the labels for the Ultrax Typically Developing dataset
uxtd_df = load_ultrasuite_labels('data/ultrasuite/labels-uxtd-uxssd-upx/', 'uxtd')

In [26]:
# Preview the data
uxtd_df.head()

,start_time,end_time,utterance,dataset,speaker,session,speech_waveform
0,00:00:07.300000,00:00:08.180000,watch,uxtd,37M,None,001A
1,00:00:08.270000,00:00:09.219999,fishing,uxtd,37M,None,001A
2,00:00:09.539999,00:00:10.500000,gloves,uxtd,37M,None,001A
3,00:00:10.640000,00:00:11.520000,spider,uxtd,37M,None,001A
4,00:00:01.170000,00:00:02.020000,r,uxtd,30F,None,010B


In [25]:
# Preview the dataframe info
uxtd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6094 entries, 0 to 6093
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   start_time       6094 non-null   timedelta64[ns]
 1   end_time         6094 non-null   timedelta64[ns]
 2   utterance        6094 non-null   object         
 3   dataset          6094 non-null   object         
 4   speaker          6094 non-null   object         
 5   session          0 non-null      object         
 6   speech_waveform  6094 non-null   object         
dtypes: object(5), timedelta64[ns](2)
memory usage: 333.4+ KB


In [24]:
# Load the labels for the Ultraphonix dataset
upx_df = load_ultrasuite_labels('data/ultrasuite/labels-uxtd-uxssd-upx/', 'upx')

In [27]:
# Preview the data
upx_df.head()

,start_time,end_time,utterance,dataset,speaker,session,speech_waveform
0,00:00:00,00:00:01.400000,sigh,upx,20M,Post,012A
1,00:00:01.639999,00:00:02.910000,sausages,upx,20M,Post,012A
2,00:00:03.140000,00:00:03.970000,snail,upx,20M,Post,012A
3,00:00:04.890000,00:00:05.699999,beige,upx,20M,Post,012A
4,00:00:00.510000,00:00:01.240000,sack,upx,16M,BL3,016A


In [33]:
def extract_segments(y, sr, segments):
    # compute segment regions in number of samples
    starts = np.floor(segments.start_time.dt.total_seconds() * sr).astype(int)
    ends = np.ceil(segments.end_time.dt.total_seconds() * sr).astype(int)
    
    file_path = 'data/transformed/'
    
    i = 0
    # slice the audio into segments
    for start, end in zip(starts, ends):
        audio_seg = y[start:end]
        print('extracting audio segment:', len(audio_seg), 'samples')
        
        file_name = file_path + str(segments.utterance[i]) + ".wav"
        
        sf.write(file_name, audio_seg, sr)
        i += 1

In [34]:
path = 'data/ultrasuite/core-uxssd/core/01M/BL1/001A.wav'
y, sr = librosa.load(path, sr=22050)
extract_segments(y, sr, uxssd_df)

extracting audio segment: 15435 samples
extracting audio segment: 19625 samples
extracting audio segment: 18964 samples
extracting audio segment: 19846 samples
extracting audio segment: 11246 samples
extracting audio segment: 12790 samples
extracting audio segment: 11026 samples
extracting audio segment: 12790 samples
extracting audio segment: 3970 samples
extracting audio segment: 10144 samples
extracting audio segment: 7719 samples
extracting audio segment: 12128 samples
extracting audio segment: 11467 samples
extracting audio segment: 16979 samples
extracting audio segment: 16317 samples
extracting audio segment: 15877 samples
extracting audio segment: 11025 samples
extracting audio segment: 8821 samples
extracting audio segment: 1985 samples
extracting audio segment: 11688 samples
extracting audio segment: 5292 samples
extracting audio segment: 3528 samples
extracting audio segment: 7938 samples
extracting audio segment: 10805 samples
extracting audio segment: 16097 samples
extract

extracting audio segment: 12128 samples
extracting audio segment: 10365 samples
extracting audio segment: 16317 samples
extracting audio segment: 13231 samples
extracting audio segment: 18302 samples
extracting audio segment: 20948 samples
extracting audio segment: 19625 samples
extracting audio segment: 17420 samples
extracting audio segment: 16317 samples
extracting audio segment: 20287 samples
extracting audio segment: 15876 samples
extracting audio segment: 13231 samples
extracting audio segment: 17641 samples
extracting audio segment: 19185 samples
extracting audio segment: 27784 samples
extracting audio segment: 28225 samples
extracting audio segment: 14774 samples
extracting audio segment: 13452 samples
extracting audio segment: 16538 samples
extracting audio segment: 11026 samples
extracting audio segment: 16538 samples
extracting audio segment: 8820 samples
extracting audio segment: 15436 samples
extracting audio segment: 17420 samples
extracting audio segment: 12348 samples
e

extracting audio segment: 13230 samples
extracting audio segment: 22271 samples
extracting audio segment: 15215 samples
extracting audio segment: 14554 samples
extracting audio segment: 7498 samples
extracting audio segment: 17641 samples
extracting audio segment: 17862 samples
extracting audio segment: 7939 samples
extracting audio segment: 9041 samples
extracting audio segment: 27563 samples
extracting audio segment: 20948 samples
extracting audio segment: 14113 samples
extracting audio segment: 24256 samples
extracting audio segment: 16098 samples
extracting audio segment: 1324 samples
extracting audio segment: 17861 samples
extracting audio segment: 3088 samples
extracting audio segment: 23595 samples
extracting audio segment: 28004 samples
extracting audio segment: 11026 samples
extracting audio segment: 12789 samples
extracting audio segment: 10144 samples
extracting audio segment: 13451 samples
extracting audio segment: 18081 samples
extracting audio segment: 1985 samples
extrac

extracting audio segment: 23374 samples
extracting audio segment: 22712 samples
extracting audio segment: 8820 samples
extracting audio segment: 13451 samples
extracting audio segment: 18082 samples
extracting audio segment: 11466 samples
extracting audio segment: 11466 samples
extracting audio segment: 17641 samples
extracting audio segment: 16538 samples
extracting audio segment: 18303 samples
extracting audio segment: 18081 samples
extracting audio segment: 18082 samples
extracting audio segment: 17640 samples
extracting audio segment: 12789 samples
extracting audio segment: 12569 samples
extracting audio segment: 22271 samples
extracting audio segment: 16759 samples
extracting audio segment: 11246 samples
extracting audio segment: 21830 samples
extracting audio segment: 15656 samples
extracting audio segment: 5954 samples
extracting audio segment: 20507 samples
extracting audio segment: 6616 samples
extracting audio segment: 4852 samples
extracting audio segment: 24476 samples
extr

extracting audio segment: 15215 samples
extracting audio segment: 31532 samples
extracting audio segment: 0 samples
extracting audio segment: 16097 samples
extracting audio segment: 15215 samples
extracting audio segment: 18302 samples
extracting audio segment: 10805 samples
extracting audio segment: 29769 samples
extracting audio segment: 22713 samples
extracting audio segment: 12569 samples
extracting audio segment: 9703 samples
extracting audio segment: 17421 samples
extracting audio segment: 17861 samples
extracting audio segment: 18082 samples
extracting audio segment: 24697 samples
extracting audio segment: 18082 samples
extracting audio segment: 17200 samples
extracting audio segment: 13231 samples
extracting audio segment: 22271 samples
extracting audio segment: 25358 samples
extracting audio segment: 7938 samples
extracting audio segment: 9041 samples
extracting audio segment: 12349 samples
extracting audio segment: 4851 samples
extracting audio segment: 2646 samples
extractin

...

extracting audio segment: 11026 samples
extracting audio segment: 17640 samples
extracting audio segment: 17862 samples
extracting audio segment: 16979 samples
extracting audio segment: 13892 samples
extracting audio segment: 4190 samples
extracting audio segment: 2867 samples
extracting audio segment: 9482 samples
extracting audio segment: 12570 samples
extracting audio segment: 3308 samples
extracting audio segment: 1544 samples
extracting audio segment: 14553 samples
extracting audio segment: 11908 samples
extracting audio segment: 12128 samples
extracting audio segment: 9923 samples
extracting audio segment: 6616 samples
extracting audio segment: 11247 samples
extracting audio segment: 24255 samples
extracting audio segment: 14995 samples
extracting audio segment: 23153 samples
extracting audio segment: 14333 samples
extracting audio segment: 13010 samples
extracting audio segment: 7278 samples
extracting audio segment: 14554 samples
extracting audio segment: 22933 samples
extracti

extracting audio segment: 18302 samples
extracting audio segment: 15877 samples
extracting audio segment: 15877 samples
extracting audio segment: 17200 samples
extracting audio segment: 12790 samples
extracting audio segment: 15877 samples
extracting audio segment: 1985 samples
extracting audio segment: 5513 samples
extracting audio segment: 13451 samples
extracting audio segment: 18082 samples
extracting audio segment: 14333 samples
extracting audio segment: 41675 samples
extracting audio segment: 5733 samples
extracting audio segment: 12128 samples
extracting audio segment: 17421 samples
extracting audio segment: 15656 samples
extracting audio segment: 10364 samples
extracting audio segment: 14774 samples
extracting audio segment: 13672 samples
extracting audio segment: 14334 samples
extracting audio segment: 12128 samples
extracting audio segment: 10143 samples
extracting audio segment: 12128 samples
extracting audio segment: 13231 samples
extracting audio segment: 22271 samples
ext

extracting audio segment: 12790 samples
extracting audio segment: 15215 samples
extracting audio segment: 18302 samples
extracting audio segment: 20508 samples
extracting audio segment: 27123 samples
extracting audio segment: 17420 samples
extracting audio segment: 19846 samples
extracting audio segment: 18303 samples
extracting audio segment: 20728 samples
extracting audio segment: 17200 samples
extracting audio segment: 20287 samples
extracting audio segment: 22491 samples
extracting audio segment: 1338 samples
extracting audio segment: 18964 samples
extracting audio segment: 16758 samples
extracting audio segment: 16759 samples
extracting audio segment: 11467 samples
extracting audio segment: 2867 samples
extracting audio segment: 13672 samples
extracting audio segment: 5293 samples
extracting audio segment: 13893 samples
extracting audio segment: 2647 samples
extracting audio segment: 17641 samples
extracting audio segment: 7277 samples
extracting audio segment: 18964 samples
extra

extracting audio segment: 18522 samples
extracting audio segment: 19846 samples
extracting audio segment: 10144 samples
extracting audio segment: 22713 samples
extracting audio segment: 26020 samples
extracting audio segment: 12128 samples
extracting audio segment: 3528 samples
extracting audio segment: 8820 samples
extracting audio segment: 13672 samples
extracting audio segment: 15876 samples
extracting audio segment: 11687 samples
extracting audio segment: 14553 samples
extracting audio segment: 15876 samples
extracting audio segment: 17420 samples
extracting audio segment: 21610 samples
extracting audio segment: 14554 samples
extracting audio segment: 20727 samples
extracting audio segment: 1324 samples
extracting audio segment: 18743 samples
extracting audio segment: 16980 samples
extracting audio segment: 16097 samples
extracting audio segment: 5072 samples
extracting audio segment: 14994 samples
extracting audio segment: 11907 samples
extracting audio segment: 9041 samples
extra

extracting audio segment: 1558 samples
extracting audio segment: 4410 samples
extracting audio segment: 6837 samples
extracting audio segment: 16759 samples
extracting audio segment: 16538 samples
extracting audio segment: 14553 samples
extracting audio segment: 9041 samples
extracting audio segment: 10365 samples
extracting audio segment: 11026 samples
extracting audio segment: 1323 samples
extracting audio segment: 1323 samples
extracting audio segment: 44541 samples
extracting audio segment: 10364 samples
extracting audio segment: 27122 samples
extracting audio segment: 13010 samples
extracting audio segment: 12569 samples
extracting audio segment: 13672 samples
extracting audio segment: 0 samples
extracting audio segment: 11025 samples
extracting audio segment: 9262 samples
extracting audio segment: 15877 samples
extracting audio segment: 8820 samples
extracting audio segment: 12348 samples
extracting audio segment: 16980 samples
extracting audio segment: 12790 samples
extracting a